# Resolución ejercicio 3.10.29
## Análisis y diseño de algoritmos distribuidos en redes
### Andrés Montoro 5.169.779-1

Show the step-by-step execution of Stages and of UniStages in the ring of Figure 3.3. Indicate for each step, the values know at the candidates.

Se asumen las restricciones R + ID:
- Links bidireccionales
- Conectividad
- Total reliability
- Valores iniciales distintos

Si bien Conectividad no es necesaria pues la topologia la restringimos a un anillo de n nodos como indica la figura 3.3:
- Anillo

In [ ]:
from pydistsim.algorithm.node_algorithm import NodeAlgorithm, StatusValues
from pydistsim.algorithm.node_wrapper import NodeAccess
from pydistsim.message import Message
from pydistsim.restrictions.communication import BidirectionalLinks
from pydistsim.restrictions.reliability import TotalReliability
# from pydistsim.restrictions.communication import MessageOrdering
from pydistsim.restrictions.topological import Connectivity
from pydistsim.restrictions.knowledge import InitialDistinctValues
import random
#TODO Ring??


## Stages

### Implementacion

In [ ]:
class Stages(NodeAlgorithm):
    default_params = {
        "election" : "Election",
        "notify" : "Notify",
    }

    class Status(StatusValues):
        ASLEEP = "ASLEEP",
        CANDIDATE = "CANDIDATE",
        WAITING = "WAITING",
        DEFEATED = "DEFEATED",
        FOLLOWER = "FOLLOWER",
        LEADER = "LEADER"

    S_init = (Status.ASLEEP)
    S_term = (Status.FOLLOWER, Status.LEADER)


    algorithm_restrictions = (
        BidirectionalLinks, Connectivity, TotalReliability, InitialDistinctValues
    )

    def initialize(self, node: NodeAccess):
        node.memory["stage"] = 1
        node.memory["count"] = 0
        node.memory["min"] = node.id
        self.send(
            node,
            data= [node.id, node.memory["stage"]],
            destination= list(node.neighbors),
            header= self.default_params["election"],
        )
        pass

    def error(self, method : str, message: Message):
        msj = 'Unexpected message in ' + method + message.header + " from " + str(message.source) + " , content: " + str(message.data)
        raise Exception(msj)


    def initializer(self):
        for node in self.network.nodes():
            node.status = self.Status.ASLEEP
            node.memory["min"] = None
            node.memory["stage"] = 0
            node.memory["count"] = 0
            # node.push_to_inbox(Message(meta_header=NodeAlgorithm.INI, destination=node))


    @Status.ASLEEP
    def spontaneously(self, node: NodeAccess, message: Message):
        self.initialize()
        node.status = self.Status.CANDIDATE


    @Status.ASLEEP
    def receiving(self, node: NodeAccess, message: Message):
        self.initialize()
        if message.header == self.default_params["election"]:
            id_ = message.data[1]
            min = min(node.id, id_)
            # close(sender)
            node.status = self.Status.WAITING
        else:
            self.error()


    @Status.CANDIDATE
    def receiving(self, node: NodeAccess, message: Message):
        if message.header == self.default_params["election"]:
            id_ = message.data[1]
            if id_ != node.id:
                min = min(node.id, id_)
                # close(sender) 
                node.status = self.Status.WAITING
            else:
                self.send(
                    node,
                    data= None,
                    destination= list(node.neighbors),
                    header= self.default_params["notify"],
                )
                node.status = self.Status.LEADER
        else:
            self.error()


    @Status.WAITING
    def receiving(self, node: NodeAccess, message: Message):
        if message.header == self.default_params["election"]:
            # open(other)
            node.memory["stage"] += 1
            id_ = message.data[0]
            node.memory["min"] = min(node.id, id_)
            if node.memory["min"] == node.id:
                self.send(
                    node,
                    data= [node.id, node.memory["stage"]],
                    destination= list(node.neighbors),
                    header= self.default_params["election"],
                )
                node.status = self.Status.CANDIDATE
            else:
                node.status = self.Status.DEFEATED
        else:
            self.error()
    

    @Status.DEFEATED
    def receiving(self, node: NodeAccess, message: Message):
        self.send(
            node,
            data= message.data,
            destination=[other for other in node.neighbors if other != message.source],
            header= message.header,
        )
    
    @Status.LEADER
    def default(self, *args, **kwargs):    
        pass

    @Status.FOLLOWER
    def default(self, *args, **kwargs):    
        pass

### Ejecución


In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt
from pydistsim import NetworkGenerator, Simulation
from pydistsim.logging import set_log_level, LogLevels, enable_logger
from pydistsim.network.behavior import NetworkBehaviorModel
from pydistsim.gui import drawing as draw


net_gen = NetworkGenerator(100, directed=False)
net = net_gen.generate_random_network()
sim = Simulation(net, check_restrictions=True)
sim.algorithms = (Stages,)
set_log_level(LogLevels.INFO)
enable_logger()

behaviour = NetworkBehaviorModel(message_ordering=True, message_loss_indicator=None, )
sim.network.behavioral_properties = behaviour

fig = draw.draw_current_state(sim)
fig

sim.run()
#fig = draw.draw_current_state(sim)
#fig

values = []
for node in net.nodes():
    values.append((node.memory["rank"], node.memory["value"]))

piso = -1
for (rank, val) in sorted(values):
    if piso > val:
        assert False, "Error en el ranking: " + str(val)
    piso = val



## UniStages


### Implementacion

In [ ]:
class UniStages(NodeAlgorithm):
    default_params = {
        "election" : "Election",
        "notify" : "Notify",
    }

    class Status(StatusValues):
        ASLEEP = "ASLEEP",
        CANDIDATE = "CANDIDATE",
        DEFEATED = "DEFEATED",
        FOLLOWER = "FOLLOWER",
        LEADER = "LEADER"

    S_init = (Status.ASLEEP)
    S_term = (Status.FOLLOWER, Status.LEADER)


    algorithm_restrictions = (
        Connectivity, TotalReliability, InitialDistinctValues #BidirectionalLinks, 
    )

    def initialize(self, node: NodeAccess):
        node.memory["stage"] = 1
        node.memory["count"] = 0
        node.memory["order"] = 0
        node.memory["value1"] = node.id
        self.send(
            node,
            data= [node.memory["value1"], node.memory["stage"], node.memory["order"]],
            destination= list(node.neighbors),
            header= self.default_params["election"],
        )

    def process_message(self, node: NodeAccess, value_, stage_, order_):
        if stage_ == node.memory["stage"]:
            if order_ == 0:
                envelope = value_
                order = 1;
                self.send(
                    node,
                    data= [value_, stage_, order],
                    destination= list(node.neighbors),
                    header= self.default_params["election"],
                )
            else:
                value2 = value_
            count =count+1;
            if count == 2:
                if envelope < min(value1, value2):
                    order = 0;
                    count = 0;
                    stage = stage+1;
                    value1 = envelope;
                    self.send(
                        node,
                        data= [value1, stage, order],
                        destination= list(node.neighbors),
                        header= self.default_params["election"],
                    )
                else:
                    node.status = self.Status.DEFEATED
        elif stage_ > node.memory["stage"]:
            self.send(
                node,
                data= [value_, stage_, order_],
                destination= list(node.neighbors),
                header= self.default_params["election"],
            )
            node.status = self.Status.DEFEATED;

    def error(self, method : str, message: Message):
        msj = 'Unexpected message in ' + method + message.header + " from " + str(message.source) + " , content: " + str(message.data)
        raise Exception(msj)


    def initializer(self):
        for node in self.network.nodes():
            node.status = self.Status.ASLEEP
            node.memory["stage"] = None
            node.memory["count"] = None
            node.memory["order"] = None
            node.memory["value1"] = None
            node.memory["value2"] = None
            # node.push_to_inbox(Message(meta_header=NodeAlgorithm.INI, destination=node))


    @Status.ASLEEP
    def spontaneously(self, node: NodeAccess, message: Message):
        self.initialize()
        node.status = self.Status.CANDIDATE


    @Status.ASLEEP
    def receiving(self, node: NodeAccess, message: Message):
        if message.header == self.default_params["election"]:
            self.send(
                node,
                data= message.data,
                destination= list(node.neighbors),
                header= self.default_params["election"],
            )
            node.status = self.Status.DEFEATED
        else:
            self.error()


    @Status.CANDIDATE
    def receiving(self, node: NodeAccess, message: Message):
        if message.header == self.default_params["election"]:
            value_ = message.data[0]
            if value_ != node.memory["value1"]:
                self.process_message(node, value_, message.data[1], message.data[2])
            else:
                self.send(
                    node,
                    data= None,
                    destination= list(node.neighbors),
                    header= self.default_params["notify"],
                )
                node.status = self.Status.LEADER
        else:
            self.error()

    @Status.DEFEATED
    def receiving(self, node: NodeAccess, message: Message):
        self.send(
            node,
            data= message.data,
            destination=[other for other in node.neighbors if other != message.source],
            header= message.header,
        )
        if message.header == self.default_params["notify"]:
            node.status = self.Status.FOLLOWER
    
    @Status.LEADER
    def default(self, *args, **kwargs):    
        pass

    @Status.FOLLOWER
    def default(self, *args, **kwargs):    
        pass

### Ejecucion

## Otros

In [ ]:
sim.reset()
plt.close()